In [16]:
!pip install datasets
## if GPU installed
#!pip install torch torchvision torchaudio
# if not
!pip install torch==1.10.0+cpu torchvision==0.11.1+cpu torchaudio==0.10.0+cpu -f https://download.pytorch.org/whl/torch_stable.html
# need tranformer library as well
!pip install transformers

!pip install sentencepiece
!pip install accelerate
!pip install rouge_score
!pip install evaluate

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.10.0+cpu (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu102, 1.11.0+cu113, 1.11.0+cu115, 1.11.0+rocm4.3.1, 1.11.0+rocm4.5.2, 1.12.0, 1.12.0+cpu, 1.12.0+cu102, 1.12.0+cu113, 1.12.0+cu116, 1.12.0+rocm5.0, 1.12.0+rocm5.1.1, 1.12.1, 1.12.1+cpu, 1.12.1+cu102, 1.12.1+cu113, 1.12.1+cu116, 1.12.1+rocm5.0, 1.12.1+rocm5.1.1, 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.0+rocm5.1.1, 1.13.0+rocm5.2, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 1.13.1+rocm5.1.1, 1.13.1+rocm5.2, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.0+rocm5.3, 2.0.0+rocm5.4.2, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2, 2.1.0, 2.1.0+cpu, 2.1.0+cpu.cxx11.abi, 2.1.0+cu118, 2.1.0+cu121, 2.1

In [4]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=da521dcf9cabfda2188155157608954fa13a3c6925ef10e07bc79c62f54e2afa
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [25]:
from datasets import load_dataset
import ipywidgets as widgets
from IPython.display import display
import torch
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from rouge_score import rouge_scorer


# Load dataset
ds = load_dataset("Kaludi/Customer-Support-Responses")

from datasets import load_dataset
from sklearn.model_selection import train_test_split
from datasets import load_dataset  # Import load_dataset
import ipywidgets as widgets
from IPython.display import display
import torch
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from rouge_score import rouge_scorer

# Load dataset from the hub
# Load dataset
# Load dataset from the hub
dataset = load_dataset("Kaludi/Customer-Support-Responses")

print(f"Total dataset size: {len(dataset['train'])}")

# Split into train and test sets (combined logic)
train_data, test_data = train_test_split(dataset['train'], test_size=0.30, random_state=42)

# Print sizes of train and test datasets
print(f"Train dataset size: {len(train_data)}")
print(f"Test dataset size: {len(test_data)}")



# Load pre-trained model and tokenizer
model_name = 'facebook/blenderbot-400M-distill'
tokenizer = BlenderbotTokenizer.from_pretrained(model_name)
model = BlenderbotForConditionalGeneration.from_pretrained(model_name)

# Ensure the model uses the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Ensure the model uses the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to generate response using seq2seq model
def generate_response(query):
    # Tokenize the input query
    inputs = tokenizer(query, return_tensors="pt", max_length=512, truncation=True).to(device)

    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'].to(device),
            max_length=100,
            num_beams=4,
            length_penalty=2.0,
            early_stopping=True
        )

        # Decode and return the generated response
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generated_text

# Widgets for input, button, and output
input_text = widgets.Textarea(
    value='',
    placeholder='Type your query here',
    description='Query:',
    disabled=False
)
output_text = widgets.Textarea(
    value='',
    placeholder='Response will appear here',
    description='Response:',
    disabled=True
)
button = widgets.Button(description="Generate Response")

# Function to handle button click event
def on_button_click(b):
    query = input_text.value.strip()
    if query:
        response = generate_response(query)
        output_text.value = response
    else:
        output_text.value = "Please enter a query."

# Attach the click event handler to the button
button.on_click(on_button_click)

# Display widgets
display(widgets.VBox([input_text, button, output_text]))


Total dataset size: 74
Train dataset size: 2
Test dataset size: 2


In [26]:
import evaluate

# Load the ROUGE evaluation metric
rouge = evaluate.load('rouge')


test_data = ["where is my order", "Oh no! What did you order? I hope it wasn't too"]

# Compute the ROUGE score
results = rouge.compute(predictions=test_data, references=test_data)

# Print the results
print(results)

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}
